# Algorithms for Genomic Data Science--Week 3

In [1]:
def editDistance(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0]*(len(y)+1))
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    for i in range(1, len(x)+1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    return D[-1][-1]

In [2]:
alphabet = ['A', 'C', 'G', 'T']
score = [[0, 4, 2, 4, 8], \
        [4, 0, 4, 2, 8], \
        [2, 4, 0, 4, 8], \
        [2, 4, 2, 0, 8], \
        [8, 8, 8, 8, 8]]

In [3]:
def globalAlignment(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0]*(len(y)+1))
    for i in range(1, len(x)+1):
        D[i][0] = D[i-1][0] + 1 + score(alphabet.index(x[i-1]))[-1] #gives current char to necessary score
    for i in range(len(y)+1):
        D[0][i] = D[0][i-1] + score[-1][alphabet.index(y[i-1])]
    for i in range(1, len(x)+1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]
            distVer = D[i-1][j] + score[alphabet.index(x[i-1])][-1]
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]
            D[i][j] = min(distHor, distVer, distDiag)
    return D[-1][-1]

In [4]:
def overlap(a, b, min_length=3):
    start = 0
    
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1

In [5]:
overlap("TTACGT", 'CGTACCGT')

3

In [6]:
from itertools import permutations
list(permutations([1, 2, 3], 3))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [7]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [8]:
reads = ['ACGGTGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGTGATC', 'GATCAAGT'): 4}


## Exercises

#### Question 1
What is the edit distance of the best match between pattern 'GCTGATCGATCGTACG' and the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [9]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [10]:
chrom = readGenome('chr1.GRCh38.excerpt.fasta')
print(chrom)

...


In [11]:
def HammingDistance(p, q):
    count = 0
    for i, j in zip(p, q):
        if i != j:
            count += 1
    return count

In [12]:
def ApproximatePatternMatching(Text, Pattern, d):
    positions = []
    for i in range(len(Text)-len(Pattern) + 1):
        if HammingDistance(Text[i:i+len(Pattern)], Pattern) <= d:
            positions.append(i)
    return positions

In [13]:
p = 'GCTGATCGATCGTACG'
ApproximatePatternMatching(chrom, p, 3)

[380536]

In [14]:
editDistance(chrom[380536:380552], p)

3

#### Question 2
What is the edit distance of the best match between pattern 'GATTTACCAGATTGAG' and the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [15]:
p = 'GATTTACCAGATTGAG'
ApproximatePatternMatching(chrom, p, 3)

[92165, 271664]

In [16]:
ApproximatePatternMatching(chrom, p, 2)

[]

In [17]:
editDistance(p, chrom[92165:92181])

3

In [18]:
chrom[271664:271680]

'GTTTTACCACAGTGAG'

In [19]:
editDistance(p, chrom[271664:271680])

3

#### Question 4
Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. Ignore reverse complements.

In [20]:
def readFastq(filename):
    sequences =[]
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [22]:
seqs, quals = readFastq('ERR266411_1.for_asm.fastq')

In [23]:
def CreatingKmers(seqs, k):
    kmers = {}
    for read in seqs:
        n = len(read)
        for i in range(n-k+1):
            kmer = read[i:i+k]
            kmers.update({kmer : read})
    return kmers

In [24]:
KmerDict = CreatingKmers(seqs, 30)

In [25]:
len(KmerDict)

108344

In [26]:
from collections import defaultdict
def overlap_all_pairs(reads, min_length = 30):
    
    kmer_dict = KmerDict #createKmersFromReads(reads, min_length) #create dict with kmer key/and set of reads with that kmer value. User has to create this function

    overlap_graph = defaultdict(set) #read is key, set of overlapping reads are value    
    
    for read in reads:
        #create suffix for this read
        read_suffix = read[-min_length:]
        
        #extract set of all reads containing this kmer/suffix
        read_set = kmer_dict[read_suffix]
        
        assert(len(read_set) > 0) # check that the set isnt empty
        
        read_set.remove(read) #remove the read so we dont compare it with itself
        

        #THIS WORKS
        for compare_read in read_set:
            if overlap(read, compare_read, 30):
                overlap_graph[read].add(compare_read)

In [28]:
print(overlap_all_pairs(reads, min_length=30))

KeyError: 'ACGGTGATC'

In [ ]:
naive_overlap_map(KmerDict, 30)

In [ ]:
print(KmerDict)